In [ ]:
%pip install monotonic openpyxl

In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))

# Import neo4j DB: 1/?

Code to translate v2.7.4_PIS-model.xlsx to neo4j database. 

## Setup

In [ ]:
import pandas as pd
import re
import numpy as np
import os
from IPython.display import Image, display

In [ ]:
import helpers

In [ ]:
from importlib import reload

In [ ]:
node_labels = helpers.node_labels

## Read in sheets

In [ ]:
from pathlib import Path

base_path = Path("..")
input_path = base_path / "data" / "raw"
output_path = base_path / "data" / "parsed"

### Components sheet

In [ ]:
sheets = [#(file, sheet_name)]
    ("v2.7.9_PIS-model.xlsx", "Components"),
#    ("v2.7.5_PIS-model.xlsx", "Components_New"), 
#    ("Model_CK.xlsx", "Components_new"), 
#    ("v2.7.2_PIS-model-JALR.xlsx", "Components_New")
]

In [ ]:
# resave xlsx as tsv
drops = ['Legacy:Process', 'Notes', 'Unnamed: 21', 'GMM_OCD', 'ExternalDB', 'Notes', 'GMM:Synonyms.1']
col_rename = {
    'mID':'identifier',
    'AddedBy':'AddedBy', 
    'Species':'Species', 
    'NodeType':'NodeType', 
    'Family':'Family', 
    'Clade':'Clade', 
    'NodeID':'NodeID', 
    'NodeName':'NodeName', 
    'ModelStatus':'ModelStatus', 
    'NodeDescription':'NodeDescription', 
    'AdditionalInfo':'AdditionalInfo', 
    'Process':'Process', 
    'ModelV':'ModelV', 

    'ExtDBlink':'ExtDBlink', 

    'GMM_OCD1':'GMM_OCD', 
    'GMM:Description':'GMM_Description', 
    'GMM:ShortName':'GMM_ShortName', 
    'GMM:Synonyms':'synonyms', 

    'Node':'NodeName'
}

for file_name, sheet_name in sheets:   
    file_path = input_path / file_name
    base_name = file_path.stem
    
    new_file_path = output_path / f'{base_name}-{sheet_name}.tsv'
    
    print(f"{file_name} sheet '{sheet_name}' will be saved to '{new_file_path}'")
#    if os.path.exists(new_file_path):
#        continue
        
    df = pd.read_excel(file_path, 
                    sheet_name=sheet_name, 
                    header=[1], 
                    dtype=str, 
                    na_values=helpers.empty_strings)
    
    to_drop = list(set(drops) & set(df.columns)) + list(df.filter(regex=("Unnamed.*")).columns)
    df.drop(to_drop, axis=1, inplace=True)
    
    new_cols = [col_rename[x] for x in df.columns]
    df.columns = new_cols
    
    df.to_csv(new_file_path, sep="\t", index=None)

In [ ]:
dfs = []

for file_name, sheet_name in sheets:
    print(file_name, sheet_name)
    base_name, extension = os.path.splitext(file_name)
    file_path = output_path / f'{base_name}-{sheet_name}.tsv'
    df = pd.read_csv(file_path, sep="\t")
    dfs.append(df)

In [ ]:
df_components = pd.concat(dfs, sort=False)
df_components.reset_index(drop=True, inplace=True)

In [ ]:
pd.value_counts(df_components['NodeType'])

In [ ]:
df_components.shape[0]

In [ ]:
for c in df_components.columns:
    df_components[c] = df_components[c].str.strip()

In [ ]:
# df_components[df_components['NodeType']=='x']
# x = df_components[df_components['NodeType']=='x'].index; display(x)
# df_components.drop(x, inplace=True)

In [ ]:
#df_components.loc[df_components["NodeName"].isna(), 'NodeName']
#df_components.loc[df_components["NodeName"].isna(), 'NodeName'] =  df_components.loc[df_components["NodeName"].isna(), 'NodeID']

In [ ]:
components_node_type_to_node_label = {
    "plant coding":"PlantCoding",
    "plant_coding":"PlantCoding",
    "plant_noncoding":"PlantNonCoding",
    "plant_ncRNA":"PlantNonCoding",

    "plant_complex":"Complex", 

    "metabolite":"Metabolite",

    "pathogen_coding":"ForeignCoding",
    "pathogen_noncoding":"ForeignNonCoding",
    
    "plant_abstract":"PlantAbstract",
    
    "process":"Process", 

    np.nan:"Undefined"
}

In [ ]:
# update node labels
df_components["NodeLabel"] = df_components["NodeType"].apply(lambda x: components_node_type_to_node_label[x])
pd.value_counts(df_components['NodeLabel'])

In [ ]:
df_components[df_components["NodeLabel"] == "Undefined"]

In [ ]:
df_components.sort_values(["NodeLabel", "Family", "NodeName"], inplace=True)

In [ ]:
def only_asci(x):
    return "".join([character for character in x if character.isascii()])

In [ ]:
reload(helpers)

In [ ]:
df_components['species'] = df_components["Species"].apply(lambda x: x.lower())
#df_components['observed_species'] = df_components["Species"].apply(helpers.get_second_item)
#df_components['also_observed_in'] = df_components["Species"].apply(helpers.rest_of_items)

df_components["synonyms"] = df_components["synonyms"].apply(helpers.list_string_to_nice_string)
df_components['AdditionalInfo'].fillna('', inplace=True)
df_components.loc[df_components['ModelV'].isna(), 'ModelV'] = 'vNA'

df_components['GMM_OCD'].fillna('', inplace=True)
df_components['ExtDBlink'].fillna('', inplace=True)

In [ ]:
df_components['species'].unique()

In [ ]:
df_components.loc[df_components['AddedBy'].isna(), 'AddedBy']

In [ ]:
df_components['AddedBy'] = df_components['AddedBy'].apply(lambda x: x.upper())
df_components.loc[df_components['AddedBy']=='ZR/MZ', 'AddedBy'] = 'MZ' 
df_components['AddedBy'].unique()

In [ ]:
re_ec = "ec(?:\:|\s)?(\d+(?:\.(?:\-|\d+)){1,3}(?:\.n\d+)?)(?:\s|$|\]|,|\.)"

# also use "AdditionalInfo", "NodeDescription"
def get_external_links(row):
    dbs_list = []
    
    ################
    if row['Family'] == "R-gene":
        dbs_list.append("invented:unidentified")
    
    ################
    if row["NodeName"] in ["X1", "X2", "X3", "X4"]:
        dbs_list.append("invented:unidentified")
    
    ################
    ocd_id = row['GMM_OCD']
    if ocd_id:
        dbs_list.append(f"gmm_ocd:{ocd_id.lower()}")

    ################
    x = row['ExtDBlink']
    x = x.lower()
    
    pubchem_match = re.findall("(?:pubchem:)\s*(.+?)(?:\s|$)", x)
    dbs_list += [f"pubchem:{idf}" for idf in pubchem_match]
    chebi_match = re.findall("(?:chebi:)\s*(.+?)(?:\s|$)", x)
    dbs_list += [f"chebi:{idf}" for idf in chebi_match]

    # regex ocd from x as well
    ocd_match = re.findall("(ocd_all_.+?(?:\s|$))", x)
    dbs_list += [f"gmm_ocd:{idf}" for idf in ocd_match]

    go_match = re.findall("go:(\d{7})", x)
    dbs_list += [f"go:{idf.strip().rstrip(',.')}" for idf in go_match]     

    
    doi_match = re.findall("https://doi.org/(\S+)", x)
    dbs_list += [f"doi:{idf.strip().rstrip(',.')}" for idf in doi_match]     
    
    uniprot_match = re.findall("UNIPROT:([OPQ][0-9][A-Z0-9]{3}[0-9]|[A-NR-Z][0-9](?:[A-Z][A-Z0-9]{2}[0-9]){1,2})", x.upper())
    dbs_list += [f"uniprot:{idf}" for idf in uniprot_match]
                 
    mirbase_match = re.findall("mirbase:(.*)", x)
    dbs_list += [f"mirbase:{idf}" for idf in mirbase_match]

    conceptual_matches = re.findall("(conceptual:(?:process|complex))", x)
    dbs_list += [f"{idf}" for idf in conceptual_matches]
                 
    ################
    x = row['AdditionalInfo']
    x = x.lower()
    
    # EC:3.3.3.- EC 2.2.1.7
    ec_match = re.findall(re_ec, x)
    dbs_list += [f"ec:{idf}" for idf in ec_match]
    
    pubmed_match = re.findall("pmid(?:\:|)\s*(\d+)", x)
    dbs_list += [f"pmid:{idf}" for idf in pubmed_match]
    
    pubmedc_match =  re.findall("pmcid(?:\:|)\s*(pmc\d+)", x)
    dbs_list += [f"pmcid:{idf}" for idf in pubmedc_match]
    
    doi_match = re.findall("doi(?:\:|\/)\s*(.+?)(?:\s|$|:)", x)
    dbs_list += [f"doi:{idf.strip().rstrip(',.')}" for idf in doi_match]    
    
    kegg_match = re.findall(r"((?:k|map|ko|ec|rn|ath)\d{5})", x)
    dbs_list += [f"kegg:{idf}" for idf in kegg_match]              
    
    ncbi_nuccore_match = re.findall("NCBI ID: (.+)", x)
    dbs_list += [f"ncbi_nuccore:{idf}" for idf in ncbi_nuccore_match]
                 
    ################                 
    x = row['NodeDescription']
    x = x.lower()
    
    # EC:3.3.3.-
    ec_match = re.findall(re_ec, x)
    dbs_list += [f"ec:{idf}" for idf in ec_match]
     
    kegg_match = re.findall(r"((?:k|map|ko|ec|rn)\d{5})", x)
    dbs_list += [f"kegg:{idf}" for idf in kegg_match]              
    
    return ','.join(list(set(dbs_list)))
    


In [ ]:
x = df_components.apply(get_external_links, axis=1)

In [ ]:
# checks
for v in ['ETR1', \
          'GST1', \
          'NDB3', \
#           x[df_components['Clade']=='NPH3'], \
          'RBX1B', \
          'DXPS3', \
          'ACX4', \
          'phasiRNA931', \
          'ribosome',  \
          'D53', \
          'miR159b', \
          'Trichome-initiation', \
          #x[:, df_components['NodeName']=='WD/bHLH/MYB']
         ]:

    print(x[df_components[df_components['NodeName']==v].index[0]])

In [ ]:
df_components['external_links'] = x

In [ ]:
def get_db(external_links, search_db="kegg"):
    if external_links != "":
        for dbval in external_links.split(","):
            try:
                db, val = dbval.split(":")
                if db == search_db:
                    return val
            except ValueError:
                print("issue", dbval)
                return ""
    return ""

In [ ]:
df_components['gmm_ocd'] = x.apply(get_db, search_db="gmm_ocd")

In [ ]:
df_components[["NodeLabel", "Family", "Clade", "NodeName",  "NodeID", "synonyms", \
               "external_links", "gmm_ocd", "ExtDBlink", "GMM_OCD", \
               "AdditionalInfo", "NodeDescription"]].to_csv(os.path.join("..", "data", "parsed", "components-lit-check.tsv"), sep="\t", index=None)

In [ ]:
# See https://unicode-table.com/en/ and https://www.utf8-chartable.de/unicode-utf8-table.pl 

def only_asci(x):
    return "".join([character for character in x if character.isascii()])

def find_non_ascii(x):
    x = str(x)
    has_nonascii = False
    for character in x:
        if not character.isascii():
            has_nonascii = True
            print(character, ord(character), character.encode())
    if has_nonascii:
        print(x)
        return True
    else:
        return False

ascii_replacers = {
    b'\xc2\xa0'         : b' ',            # funky WIN whitespace
    b'\xe2\x80\xa6'     : b'...',          # …
    b'\xe2\x80\x8b'     : b'',             # have no figging clue
    b'\xe2\x80\x93'     : b'-',            # –
    
    b'\xce\xb1'         : b'&alpha;',      # α
    b'\xc3\x9f'         : b'&beta;',       # ß
    b'\xce\xb2'         : b'&beta;',       # β
    
    # some "prime" symbols...
    b'\xe2\x80\x98'     : b'&prime;',      # ‘ Left Single Quotation Mark
    b'\xe2\x80\x99'     : b'&prime;',      # ’ Right Single Quotation Mark
    b'\xc2\xb4'         : b'&prime;',      # ´ Acute Accent
    # actual prime
    b'\xe2\x80\xb2'     : b'&prime;',      # ′ Prime
    
    # Sorry accents :(
    b'\xc5\xa0'         : b'S',            # Š
    b'\xc5\xa1'         : b's',            # š
    b'\xc5\xbd'         : b'Z',            # Ž
    b'\xc4\x8d'         : b'c'             # č
}


def replacer(x, verbose=False):
    if type(x) == float:
        return x
    y = x.encode('utf-8')
    for old, new in ascii_replacers.items():
        y = y.replace(old, new)
    y = y.decode('utf-8')
    
    if verbose and ( y != x):
        print(f"'{x}' : '{y}'")
    
    return y.strip()


In [ ]:
bad_cols = []
for c in df_components.columns:
    print(c, "\n-------------")
    if any(df_components[c].apply(find_non_ascii)):
        bad_cols.append(c)
    print()

In [ ]:
for c in bad_cols:
    print(c, "\n-------------")
    df_components[c] = df_components[c].apply(replacer, verbose=True)
    print()

In [ ]:
for c in bad_cols:
    print(c, "\n-------------")
    if any(df_components[c].apply(find_non_ascii)):
        print()

In [ ]:
# also need to look for ' (quote) and " (doube; quote)
# which are used instead of prime, and may cause string issues

# replacing 5' with 5&prime; and 3' with 3&prime;
def find_quotes(x):
    x = str(x)
    if x.find('"') != -1: 
        print(x)
        return True
    elif x.find("'") != -1:
        print(x)
        return True
    else:
        return False
        

def quote_replacer(x, verbose=False):
    if type(x) == float:
        x = ""
    y = x.replace("5'",  "5&prime;")
    y = y.replace("3'",  "3&prime;")

    if verbose and ( y != x):
        print(f"'{x}' : '{y}'")
        
    return y

In [ ]:
bad_cols = []
for c in df_components.columns:
    print(c, "\n-------------")
    if any(df_components[c].apply(find_quotes)):
        bad_cols.append(c)
    print()

In [ ]:
for c in bad_cols:
    print(c, "\n-------------")
    df_components[c] = df_components[c].apply(quote_replacer, verbose=True)
    print()

In [ ]:
for c in bad_cols:
    print(c, "\n-------------")
    if any(df_components[c].apply(find_quotes)):
        print()

In [ ]:
# duplicated node names
for label, subdf in df_components.groupby('NodeLabel'):
    dups =  subdf[subdf.duplicated(['NodeName'], keep=False)]
    if dups.shape[0] > 0:
        print(label)
        display(dups.sort_values('NodeName'))

In [ ]:
df_components.loc[df_components['NodeLabel'].isin(['PlantCoding']), 'NodeID'] = df_components.loc[df_components['NodeLabel'].isin(['PlantCoding']), 'NodeID'].str.upper()

In [ ]:
df_components.columns

In [ ]:
want_columns = ['identifier', 'AddedBy', 'species', 'NodeLabel', 'NodeType', 
                'Family', 'Clade', 'NodeID', 'NodeName', 
                'external_links', 'NodeDescription', 'AdditionalInfo', 
                'Process', 'ModelV', 'ModelStatus',                 
                'gmm_ocd', 'GMM_Description', 'GMM_ShortName', 'synonyms'
               ]

In [ ]:
path =  output_path / "components.tsv"
df_components[want_columns].fillna('').to_csv(path, sep="\t", index=None)
print(path)

In [ ]:
!head $path

# END